# Backtesting Fundamentals**QuantLearn Module**: Backtesting & Scientific Method**Difficulty**: Intermediate**Time**: ~30 minutesBuild your first backtest from scratch. Learn the core components: signals, positions, and performance measurement.

In [ ]:
#@title 📦 Setupimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltnp.random.seed(42)plt.style.use('seaborn-v0_8-whitegrid')print("✓ Setup complete!")

## The Backtesting FrameworkEvery backtest has these components:1. **Data**: Historical prices/returns2. **Signal**: Trading logic (when to buy/sell)3. **Position**: Current holdings based on signals4. **Returns**: Strategy returns = position × market returns5. **Metrics**: Evaluate performance (Sharpe, drawdown, etc.)Let's build each piece.

In [ ]:
# 1. Generate sample price datan_days = 500dates = pd.date_range('2022-01-01', periods=n_days, freq='B')returns = np.random.normal(0.0003, 0.015, n_days)prices = 100 * np.cumprod(1 + returns)df = pd.DataFrame({    'Date': dates,    'Price': prices,    'Return': returns}).set_index('Date')print("Sample data:")print(df.head(10))plt.figure(figsize=(12, 4))plt.plot(df['Price'])plt.title('Simulated Stock Price')plt.ylabel('Price')plt.show()

## 2. Create a SignalLet's implement a simple **moving average crossover** strategy:- Buy (signal = 1) when fast MA > slow MA- Sell (signal = -1) when fast MA < slow MA

In [ ]:
# Calculate moving averagesfast_period = 20slow_period = 50df['MA_Fast'] = df['Price'].rolling(fast_period).mean()df['MA_Slow'] = df['Price'].rolling(slow_period).mean()# Generate signal: 1 = long, -1 = short, 0 = no positiondf['Signal'] = 0df.loc[df['MA_Fast'] > df['MA_Slow'], 'Signal'] = 1df.loc[df['MA_Fast'] < df['MA_Slow'], 'Signal'] = -1# Visualizefig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)# Price with MAsaxes[0].plot(df['Price'], label='Price', alpha=0.7)axes[0].plot(df['MA_Fast'], label=f'{fast_period}-day MA', linewidth=2)axes[0].plot(df['MA_Slow'], label=f'{slow_period}-day MA', linewidth=2)axes[0].set_ylabel('Price')axes[0].legend()axes[0].set_title('Price with Moving Averages')# Signalaxes[1].plot(df['Signal'], drawstyle='steps-post')axes[1].set_ylabel('Signal')axes[1].set_ylim(-1.5, 1.5)axes[1].set_title('Trading Signal (1=Long, -1=Short)')plt.tight_layout()plt.show()

## 3. Calculate Strategy Returns**Key formula**:$$r_{strategy,t} = position_{t-1} \times r_{market,t}$$We use yesterday's position because we can't see today's return before trading.

In [ ]:
# Position = previous day's signal (avoid look-ahead bias!)df['Position'] = df['Signal'].shift(1)# Strategy returnsdf['Strategy_Return'] = df['Position'] * df['Return']# Drop NaN rows (warmup period)df_clean = df.dropna()# Cumulative returnsdf_clean['Cumulative_Market'] = (1 + df_clean['Return']).cumprod()df_clean['Cumulative_Strategy'] = (1 + df_clean['Strategy_Return']).cumprod()# Plotplt.figure(figsize=(12, 5))plt.plot(df_clean['Cumulative_Market'], label='Buy & Hold', alpha=0.7)plt.plot(df_clean['Cumulative_Strategy'], label='MA Crossover Strategy', linewidth=2)plt.ylabel('Cumulative Return')plt.title('Strategy vs Buy & Hold')plt.legend()plt.show()

## 4. Performance Metrics

In [ ]:
def calculate_metrics(returns, periods_per_year=252):    """Calculate key performance metrics."""    # Remove NaN    returns = returns.dropna()    # Annualized return    total_return = (1 + returns).prod() - 1    n_years = len(returns) / periods_per_year    annual_return = (1 + total_return) ** (1/n_years) - 1    # Annualized volatility    annual_vol = returns.std() * np.sqrt(periods_per_year)    # Sharpe ratio (assuming 0% risk-free rate)    sharpe = annual_return / annual_vol if annual_vol > 0 else 0    # Maximum drawdown    cumulative = (1 + returns).cumprod()    running_max = cumulative.cummax()    drawdown = (cumulative - running_max) / running_max    max_drawdown = drawdown.min()    return {        'Annual Return': f"{annual_return*100:.2f}%",        'Annual Volatility': f"{annual_vol*100:.2f}%",        'Sharpe Ratio': f"{sharpe:.2f}",        'Max Drawdown': f"{max_drawdown*100:.2f}%",        'Total Return': f"{total_return*100:.2f}%"    }# Compare strategy vs marketprint("=== Strategy Performance ===")for k, v in calculate_metrics(df_clean['Strategy_Return']).items():    print(f"{k}: {v}")print("\n=== Buy & Hold Performance ===")for k, v in calculate_metrics(df_clean['Return']).items():    print(f"{k}: {v}")

## Exercise: Build Your Own BacktestImplement a **momentum strategy**:- If the 10-day return is positive, go long- If the 10-day return is negative, go short

In [ ]:
# Exercise: Implement momentum strategy# Use the same df DataFrame# TODO: Calculate 10-day momentum (sum of last 10 returns, or just 10-day return)lookback = 10df['Momentum'] = None  # Your code here# TODO: Generate signal based on momentumdf['Mom_Signal'] = None  # Your code here# TODO: Calculate strategy returnsdf['Mom_Position'] = None  # Your code heredf['Mom_Return'] = None  # Your code here# Print metrics# calculate_metrics(df['Mom_Return'].dropna())

In [ ]:
#@title 💡 Solution# Calculate 10-day momentumlookback = 10df['Momentum'] = df['Return'].rolling(lookback).sum()# Generate signaldf['Mom_Signal'] = np.where(df['Momentum'] > 0, 1, -1)# Position and returnsdf['Mom_Position'] = df['Mom_Signal'].shift(1)df['Mom_Return'] = df['Mom_Position'] * df['Return']# Resultsprint("=== Momentum Strategy ===")for k, v in calculate_metrics(df['Mom_Return'].dropna()).items():    print(f"{k}: {v}")# Plotdf_mom = df.dropna()df_mom['Cumulative_Momentum'] = (1 + df_mom['Mom_Return']).cumprod()plt.figure(figsize=(12, 5))plt.plot(df_mom['Cumulative_Market'], label='Buy & Hold', alpha=0.7)plt.plot(df_mom['Cumulative_Strategy'], label='MA Crossover', alpha=0.7)plt.plot(df_mom['Cumulative_Momentum'], label='Momentum', linewidth=2)plt.legend()plt.title('Strategy Comparison')plt.show()

## SummaryYou've built a complete backtest with:1. **Data preparation**: Prices → Returns2. **Signal generation**: MA crossover logic3. **Position management**: Shift signals to avoid look-ahead4. **Performance measurement**: Sharpe, drawdown, returns**Key pitfall avoided**: We used `shift(1)` to prevent look-ahead bias!**Next**: Common Pitfalls - learn about all the ways backtests can go wrong.